In [15]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [16]:
directorio = 'D:\AGC\Documentos\Capacitacion\Practica Python\Indigogo'
definitivo = 'D:\AGC\Documentos\Capacitacion\Practica Python\Indigogo\definitivo' 

columnas = ['close_date', 'currency', 'funds_raised_amount', 'funds_raised_percent', 'is_indemand', 'open_date', 'product_stage', 'project_id', 'title']

datos_tmp = pd.DataFrame(columns = columnas)

df = pd.DataFrame()
for f in os.listdir(definitivo):
#     print(f)
    if '.csv' in f:
        g = os.path.join(definitivo, f)
        datos = pd.read_csv(g)
        datos_tmp.close_date = datos.close_date
        datos_tmp.currency = datos.currency
        datos_tmp.funds_raised_amount = datos.funds_raised_amount
        datos_tmp.funds_raised_percent = datos.funds_raised_percent
        datos_tmp.is_indemand = datos.is_indemand
        datos_tmp.open_date = datos.open_date
        datos_tmp.product_stage = datos.product_stage
        datos_tmp.project_id = datos.project_id
        datos_tmp.title = datos.title
        
        df = df.append(pd.DataFrame(data=datos_tmp))

In [20]:
def create_db(db_host, db_name, db_user, db_pass):
    try:
        engine_db = create_engine(f"postgres://{db_user}:{db_pass}@{db_host}/{db_name}")
        # if not database_exists(engine_db.url):
        try:
            engine = create_engine(f"postgres://{db_user}:{db_pass}@{db_host}/postgres")
            conn = engine.connect()
            conn.execute("commit")
            command = f'CREATE DATABASE {db_name}'
            print(command)
            conn.execute(command)
            conn.execute("commit")
        except SQLAlchemyError as e:
            # print(e)
            print(f"DB {db_name} YA EXISTE")
        finally:
            conn.close()    
    except SQLAlchemyError as e:
        print(e)

def connect(db_host, db_name, db_user, db_pass):
    if db_host is not None:
        create_db(db_host, db_name, db_user, db_pass)
        DATABASE = {
            'drivername': 'postgresql',
            'host': db_host,
            'port': '5432',
            'username': db_user,
            'password': db_pass,
            'database': db_name
            }
        engine_db = create_engine(f"postgres://{db_user}:{db_pass}@{db_host}/{db_name}")
    return engine_db

In [18]:
df1 = df.dropna()
df1 = df1.drop(['close_date', 'currency', 'open_date', 'project_id', 'title', 'is_indemand'], axis=1)
df1 = df1.drop(df1[df1['funds_raised_amount']==0].index)
df1

,funds_raised_amount,funds_raised_percent,product_stage
4730,2732946.0,38.940700,production
4741,64513.0,1.278300,prototype
4742,40.0,0.002000,concept
4743,188.0,0.001880,concept
4752,7820.0,0.041376,prototype
...,...,...,...
1221,130.0,0.006500,concept
1226,1892.0,0.072769,concept
1230,531.0,0.068961,prototype
1232,15894.0,15.894000,concept


In [22]:
dbType = 'pgsql'
dbHost = 'localhost'
dbDatabase = 'indigogo'
dbUser = 'xxxx'
dbPassword = 'testing'
engine_db = connect(db_host=dbHost, db_name=dbDatabase, db_user=dbUser, db_pass=dbPassword)

df1.to_sql('indigogo', engine_db)

CREATE DATABASE agc
